#buy-and-hold

    buy, then never ever sell, until the end date :)

In [1]:
%matplotlib inline

Use future imports for python 3.0 forward compatibility

In [2]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

Other imports

In [3]:
import pandas as pd
import numpy as np
import datetime
from talib.abstract import *
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [4]:
symbol = '^GSPC'
#symbol = 'SPY'
capital = 100000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Define Strategy Class

In [5]:
class Strategy():
    """ strategy """

    def __init__(self, symbol, capital, start, end):
        self._symbol = symbol
        self._capital = capital
        self._start = start
        self._end = end

    def _algo(self):
        cash = self._capital
        shares = 0
        start_flag = True
        end_flag = False

        for i in range(len(self._ts.index)):

            date = self._ts.index[i]
            high = self._ts['high'][i]
            low = self._ts['low'][i]
            close = self._ts['close'][i]
            end_flag = True if (i == len(self._ts.index) - 1) else False

            if self._ts.index[i] < self._start:
                continue
            elif start_flag:
                start_flag = False
                # set start and end
                self._start = self._ts.index[i]
                self._end = self._ts.index[-1]

            # buy
            if self._tlog.num_open_trades() == 0:

                # calculate shares
                shares, cash = self._tlog.calc_shares(cash, close)

                # enter buy in trade log
                self._tlog.enter_trade(date, close, shares)
                print("{0} BUY  {1} {2} @ {3:.2f}".format(date, shares, 
                      self._symbol, close))

                # record daily balance
                self._dbal.append(date, high, low, close, shares, cash,
                                  pf.TradeState.OPEN)
            
            # sell 
            elif end_flag:

                # enter sell in trade log
                idx = self._tlog.exit_trade(date, close)
                shares = self._tlog.get_log()['qty'][idx]
                print("{0} SELL {1} {2} @ {3:.2f}".format(date, shares,
                      self._symbol, close))

                # record daily balance
                self._dbal.append(date, high, low, close, shares,
                                  cash, pf.TradeState.CLOSE)   

                # update cash
                cash = self._tlog.calc_cash(cash, close, shares)
                
                # update shares
                shares = 0

            # hold            
            else:
                self._dbal.append(date, high, low, close, shares,
                                  cash, pf.TradeState.HOLD)

    def run(self):
        self._ts = pf.fetch_timeseries(self._symbol)
        self._ts = pf.select_tradeperiod(self._ts,
                                        self._start, self._end,
                                        use_adj=True, pad=False)
        self._tlog = pf.TradeLog()
        self._dbal = pf.DailyBal()

        self._algo()

    def get_logs(self):
        """ return DataFrames """
        tlog = self._tlog.get_log()
        dbal = self._dbal.get_log()
        return tlog, dbal

    def stats(self):
        tlog, dbal = self.get_logs()
        
        stats = pf.stats(self._ts, tlog, dbal,
                         self._start, self._end, self._capital)
        return stats

Run Strategy

In [6]:
s = Strategy(symbol, capital, start, end)
s.run()

1950-01-03 00:00:00 BUY  6002 ^GSPC @ 16.66
2019-05-14 00:00:00 SELL 6002 ^GSPC @ 2834.41


Retrieve log DataFrames

In [7]:
s.tlog, s.dbal = s.get_logs()
s.stats = s.stats()

In [8]:
s.tlog.tail()

,entry_date,entry_price,long_short,qty,exit_date,exit_price,pl_points,pl_cash,cumul_total
0,1950-01-03,16.66,long,6002,2019-05-14 00:00:00,2834.41,2817.75,16912134.97,16912134.97


In [9]:
s.dbal.tail()

,high,low,close,shares,cash,state
date,,,,,,
2019-05-08,17393562.37,17245433.42,17282285.05,6002,6.68,1
2019-05-09,17261578.45,17024078.89,17230067.95,6002,6.68,1
2019-05-10,17353649.65,16957996.82,17294168.89,6002,6.68,1
2019-05-13,17046826.71,16814189.13,16876851.12,6002,6.68,1
2019-05-14,17120952.00,16926367.62,17012134.97,6002,6.68,2


In [10]:
pf.print_full(s.stats)

start                                                   1950-01-03
end                                                     2019-05-14
beginning_balance                                           100000
ending_balance                                         17012134.97
total_net_profit                                       16912134.97
gross_profit                                           16912134.97
gross_loss                                                    0.00
profit_factor                                                 1000
return_on_initial_capital                                 16912.13
annual_return_rate                                            7.69
trading_period                           69 years 4 months 11 days
pct_time_in_market                                          100.00
total_num_trades                                                 1
num_winning_trades                                               1
num_losing_trades                                             

Summary

In [11]:
metrics = ('annual_return_rate',
           'max_closed_out_drawdown',
           'drawdown_annualized_return',
           'drawdown_recovery',
           'best_month',
           'worst_month',
           'sharpe_ratio',
           'sortino_ratio',
           'monthly_std')

pf.summary(s.stats, *metrics)

,strategy
annual_return_rate,7.69
max_closed_out_drawdown,-56.78
drawdown_annualized_return,-7.39
drawdown_recovery,-1.42
best_month,23.49
worst_month,-29.56
sharpe_ratio,0.56
sortino_ratio,0.72
monthly_std,4.14
